# **Hotel Booking Cancellation Predictor: Model Deployment**

This notebook demonstrates the deployment of a machine learning model designed to predict hotel booking cancellations. By leveraging **Gradio**, an open-source library for building interactive applications, we provide a user-friendly interface for real-time predictions.

---

## 1. **Import libraries**

In [11]:
!pip install gradio


  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
Using cached urllib3-2.2.2-py3-none-any.whl (121 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.19
    Uninstalling urllib3-1.26.19:
      Successfully uninstalled urllib3-1.26.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.
google-auth 2.22.0 requires urllib3<2.0, but you have urllib3 2.2.2 which is incompatible.
spacy 3.6.1 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.


In [1]:
# Data Analysis
import numpy as np
import pandas as pd

# Libraries to deploy the ML model
import os
import joblib
import gradio as gr




## 2. **Load Model**
We load the pre-trained cancellation prediction model, which was saved in a `.joblib` file in the `models` folder.

In [4]:
# loading the final model from the disk
cancellation_predictor = joblib.load('../models/hotel_cancellation_prediction_model_v1_0.joblib')

## 3. **Define Prediction Function**

Here, we implement a function that takes various booking details as input and returns the likelihood of cancellation.

In [5]:
def predict_cancellation(lead_time, market_segment_type, avg_price_per_room, no_of_adults, 
                         no_of_weekend_nights, no_of_week_nights, no_of_special_requests, arrival_month, required_car_parking_space):

    # Dictionary of inputs
    input_data = {
        'lead_time': lead_time,
        'market_segment_type': 1 if market_segment_type == 'Online' else 0,
        'no_of_special_requests': no_of_special_requests,
        'avg_price_per_room': avg_price_per_room,
        'no_of_adults': no_of_adults,
        'no_of_weekend_nights': no_of_weekend_nights,
        'required_car_parking_space': 1.0 if required_car_parking_space == "Yes" else 0.0,
        'no_of_week_nights': no_of_week_nights,
        'arrival_month': arrival_month,
        
    }

    # Create a dataframe using the dictionary of inputs
    data_point = pd.DataFrame([input_data])

    # predicting the output and probability of the output
    prediction = cancellation_predictor.predict(data_point).tolist()
    prediction_prob = np.round(100*cancellation_predictor.predict_proba(data_point)[0][0], 2) if prediction == 1 else np.round(100*cancellation_predictor.predict_proba(data_point)[0][1], 2)

    # returning the final output
    return ("Yes", str(prediction_prob)+"%") if prediction[0] == 1 else ("No", str(prediction_prob)+"%")

## 4. **Create Interface** 

Build an interactive **Gradio** interface to collect user inputs and display prediction results.

- **Gradio Interface**: The Gradio interface is designed to be intuitive, with input fields for all necessary parameters and output fields for displaying the prediction results.

In [6]:
# Create the deployment input interface
model_inputs = [
    gr.Number(label="Lead Time (in days)"),
    gr.Dropdown(label="Market Segment Type", choices=["Online", "Offline"]),
    gr.Number(label="Average Price per Room"),
    gr.Number(label="Number of Adults"),
    gr.Number(label="Number of Weekend Nights"),
    gr.Number(label="Number of Week Nights"),
    gr.Number(label="Number of Special Requests"),
    gr.Dropdown(label="Arrival Month", choices=np.arange(1,13,1).tolist()),
    gr.Dropdown(label="Required Car Parking Space", choices=["Yes", "No"])
]

# Create the deployment output interface
model_outputs = [
    gr.Textbox(label="Will the booking be cancelled?"),
    gr.Textbox(label="Chances of Cancellation")
]

In [8]:
# Define the structure of the deployment interface and how the components will interact
demo = gr.Interface(
    fn = predict_cancellation,
    inputs = model_inputs,
    outputs = model_outputs,
    allow_flagging='never',
    title = "Hotel Booking Cancellation Predictor",
    description = "This interface will predict whether a given hotel booking is likely to be cancelled based on the details of the booking.",
)

## 5. **Deploy Model**
Now, we launch the Gradio app for users to interact with the model and view predictions.

Users will be able to input details such as lead time, market segment, and other booking attributes into the Gradio interface. The model will then predict whether the booking is likely to be canceled and provide the probability of cancellation.

In [9]:
# Deploy the model
demo.launch(inline=False, share=True, debug=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://93fe45a8b022c869ff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://93fe45a8b022c869ff.gradio.live


In [10]:
# Shut down the deployed model
demo.close()

Closing server running on port: 7860
